In [ ]:
! git clone https://github.com/dheeraj7596/CONDA.git


Cloning into 'CONDA'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 189 (delta 13), reused 31 (delta 9), pack-reused 153
Receiving objects: 100% (189/189), 65.47 MiB | 16.17 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Updating files: 100% (69/69), done.


In [ ]:
pip install transformers==4.20.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 119.0 MB/s eta 0:00:00


In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.5 MB/s eta 0:00:00


In [ ]:
%load_ext autoreload
%autoreload 2

# !rm -r tmp_abssum
!mkdir tmp_abssum
%cd tmp_abssum

/content/tmp_abssum


In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import nltk

import transformers as ts
from datasets import load_dataset, load_metric
import torch
from torch import nn

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
!mkdir cache

In [ ]:
# Upload the data
data = pd.read_csv('/content/CONDA/data/cls/imdb/train/train_13.csv')
data

,text,label,id
0,"And maybe, as Fred Sandford used to say, ""one ...",negative,197
1,"The plot was very thin, although the idea of n...",negative,419
2,"""Pixote: A Lei do Mais Fraco"" deals with what ...",positive,279
3,Previous reviewer Claudio Carvalho gave a much...,positive,491
4,This is a typical late Universal Horror flick:...,negative,135
5,In the late 1940s there was a short film serie...,positive,657
6,"First of all, the actor they have to play Jesu...",negative,917
7,"When i got this movie free from my job, along ...",negative,918
8,do not ever watch this film...it is the bigges...,negative,0
9,This is a must for All but especially African ...,positive,717


In [ ]:
data['label'].value_counts()

negative    8
positive    8
Name: label, dtype: int64

In [ ]:
DATA_PATH = "/content/CONDA/data/cls/imdb/train/train_13.csv"

In [ ]:
import pandas as pd
import random

In [ ]:
import pandas as pd
import random

def generate_combinations_with_label(dataset_path, random_seed):
  
    random.seed(random_seed)
    # Чтение данных из файла
    data = pd.read_csv(dataset_path)

    # Отделение позитивных и негативных отзывов
    pos_reviews = data[data['label'] == 'positive']['text'].reset_index(drop=True)
    neg_reviews = data[data['label'] == 'negative']['text'].reset_index(drop=True)

    # Создание списка всех возможных комбинаций отзывов
    combinations = []
    for i in range(len(pos_reviews)):
        for j in range(len(neg_reviews)):
            # Выбор случайного отзыва из оставшихся
            remaining = data[(data['text'] != pos_reviews[i]) & (data['text'] != neg_reviews[j])]
            rand_review = remaining.sample(n=1)['text'].values[0]
            rand_sentiment = remaining[remaining['text'] == rand_review]['label'].values[0]

            combinations.append((pos_reviews[i], neg_reviews[j], rand_review, rand_sentiment))

    return pd.DataFrame(combinations, columns=['positive', 'negative', 'random', 'random_sentiment'])

In [ ]:
df = generate_combinations_with_label(DATA_PATH, 27)

In [ ]:
"""

import pandas as pd
import random

def generate_combinations_with_label4(dataset_path, random_seed):
    random.seed(random_seed)

    # Чтение данных из файла
    data = pd.read_csv(dataset_path)

    # Отделение позитивных и негативных отзывов
    pos_reviews = data[data['label'] == 'positive']['text'].reset_index(drop=True)
    neg_reviews = data[data['label'] == 'negative']['text'].reset_index(drop=True)

    # Создание списка всех возможных комбинаций отзывов
    combinations = []
    for i in range(len(pos_reviews)):
        for j in range(len(neg_reviews)):
            pos_review = pos_reviews[i]
            neg_review = neg_reviews[j]

            # Выбор случайного положительного отзыва, не включенного в эту пару
            available_pos_reviews = pos_reviews.drop([i])
            rand_pos_review = available_pos_reviews.sample(n=1).reset_index(drop=True)[0]

            # Выбор случайного отрицательного отзыва, не включенного в эту пару
            available_neg_reviews = neg_reviews.drop([j])
            rand_neg_review = available_neg_reviews.sample(n=1).reset_index(drop=True)[0]

            combinations.append((pos_review, neg_review, rand_pos_review, 'positive'))
            combinations.append((pos_review, neg_review, rand_neg_review, 'negative'))

    return pd.DataFrame(combinations, columns=['positive', 'negative', 'random', 'random_sentiment'])

    """

"\n\nimport pandas as pd\nimport random\n\ndef generate_combinations_with_label4(dataset_path, random_seed):\n    random.seed(random_seed)\n\n    # Чтение данных из файла\n    data = pd.read_csv(dataset_path)\n\n    # Отделение позитивных и негативных отзывов\n    pos_reviews = data[data['label'] == 'positive']['text'].reset_index(drop=True)\n    neg_reviews = data[data['label'] == 'negative']['text'].reset_index(drop=True)\n\n    # Создание списка всех возможных комбинаций отзывов\n    combinations = []\n    for i in range(len(pos_reviews)):\n        for j in range(len(neg_reviews)):\n            pos_review = pos_reviews[i]\n            neg_review = neg_reviews[j]\n\n            # Выбор случайного положительного отзыва, не включенного в эту пару\n            available_pos_reviews = pos_reviews.drop([i])\n            rand_pos_review = available_pos_reviews.sample(n=1).reset_index(drop=True)[0]\n\n            # Выбор случайного отрицательного отзыва, не включенного в эту пару\n         

In [ ]:
#df = generate_combinations_with_label4(DATA_PATH, 27)

In [ ]:
X = ('Positive: ' + df.positive + '\n' + 'Negative: ' + df.negative + '\n' + df.random_sentiment.str.title()).values
y = df.random.values

In [ ]:
#X

In [ ]:
#y 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import datasets

# Define your data as dictionaries for each split
train_data = {
    "text": X_train,
    "label" : y_train

}

val_data = {
    "text": X_val,
    "label" : y_val
}


# Convert the dictionaries to `Dataset` objects
train_dataset = datasets.Dataset.from_dict(train_data)
val_dataset = datasets.Dataset.from_dict(val_data)

# Create a `DatasetDict` object containing the train, validation, and test sets
data= datasets.DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
})

# Print the sizes of the resulting datasets
print(data)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 51
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 13
    })
})


In [ ]:
train = data['train']
valid = data['validation']

In [ ]:
# Upload the tokenizer
tokenizer = ts.AutoTokenizer.from_pretrained('facebook/bart-base', cache_dir='cache/tokenizer')
tokenizer

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [ ]:
def tokenizing_fn(instances):
    encoded = tokenizer(instances["text"], truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(instances["label"], truncation=True)
        
    encoded["labels"] = labels["input_ids"]
    return encoded

tokenized_train = train.map(tokenizing_fn, batched=True).remove_columns(['text', 'label'])
tokenized_val = valid.map(tokenizing_fn, batched=True).remove_columns(['text', 'label'])
tokenized_train

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 51
})

In [ ]:
data_collator = ts.DataCollatorForSeq2Seq(tokenizer = tokenizer, padding="longest")
data_collator

DataCollatorForSeq2Seq(tokenizer=PreTrainedTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}), model=None, padding='longest', max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100, return_tensors='pt')

In [ ]:
pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b56b7d13230ba2953a59697f46b936bc42805bf71bfb6e58b4eb625783571ee2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.2 MB/s eta 0:00:00


In [ ]:
metric = load_metric("rouge", cache_dir="cache/metric")
additional_metrics = [load_metric("sacrebleu",  cache_dir="cache/metric")]

<ipython-input-23-75f88384530d>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge", cache_dir="cache/metric")


In [ ]:
def compute_metrics(eval_preds):
    # in this task we suppose that golden labels are reference sentences
    # and predictions are summaries generated by model, in human-readable format
    predictions, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(
        predictions, skip_special_tokens=True
    )
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    for add_metric in additional_metrics:
        result.update(
            add_metric.compute(predictions=predictions, references=labels)
        )

    return result

In [ ]:
def init_model():
    return ts.AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base', cache_dir = 'cache/model').to(device)

In [ ]:
training_args = ts.Seq2SeqTrainingArguments(
    output_dir='tmp/model_output',
    # Batch size args
    num_train_epochs=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # Optimizer args
    learning_rate=2e-5,
    weight_decay=0.028,
    max_grad_norm=0.28,
    # Scheduler args
    warmup_ratio=0.1,
    # Eval args
    metric_for_best_model='eval_rouge1',
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=3,
    # WANDB args
    report_to="wandb",  # enable logging to W&B
    run_name="bart_hyperopt",  # name of the W&B run (optional)
    # Seq2Seq args
    generation_max_length=42,
    predict_with_generate=True,
    generation_num_beams=4,
    # General args
    seed=42,
    #fp16=True,
    fp16_full_eval=False,
    gradient_accumulation_steps=8
)
callbacks = [ts.EarlyStoppingCallback(early_stopping_patience=20)]

In [ ]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=e19d6bf3495ff219f6efe592f510b8094d13ad5fb6c4952c61572a9e98d1486e
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
trainer = ts.Seq2SeqTrainer(
    model_init=init_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    callbacks=callbacks,
    compute_metrics=compute_metrics
)

https://huggingface.co/facebook/bart-base/resolve/main/config.json not found in cache or force_download set to True, downloading to /content/tmp_abssum/cache/model/tmpo9kejcw_


Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-base/resolve/main/config.json in cache at cache/model/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.a243ed957122436adb0b8d8e9d20f896f45c174b6324d625ca0a20a84f72a910
creating metadata file for cache/model/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.a243ed957122436adb0b8d8e9d20f896f45c174b6324d625ca0a20a84f72a910
loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at cache/model/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.a243ed957122436adb0b8d8e9d20f896f45c174b6324d625ca0a20a84f72a910
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-base/resolve/main/pytorch_model.bin in cache at cache/model/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.f2f355ad2775769afc60592b43a46d72ca548375e3a1d65f381a751e711cbadd
creating metadata file for cache/model/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.f2f355ad2775769afc60592b43a46d72ca548375e3a1d65f381a751e711cbadd
loading weights file https://huggingface.co/facebook/bart-base/resolve/main/pytorch_model.bin from cache at cache/model/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.f2f355ad2775769afc60592b43a46d72ca548375e3a1d65f381a751e711cbadd
All model checkpoint weights were used when initializing BartForConditionalGeneration.

All the weights of BartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
trainer.train()

loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at cache/model/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.a243ed957122436adb0b8d8e9d20f896f45c174b6324d625ca0a20a84f72a910
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpoint

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Score,Counts,Totals,Precisions,Bp,Sys Len,Ref Len
1,5.139100,4.234396,10.446112,1.385992,7.014250,9.656221,42.000000,0.229427,"[165, 0, 0, 0]","[572, 559, 546, 533]","[28.846153846153847, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
2,5.027500,4.155552,10.446112,1.385992,7.014250,9.656221,42.000000,0.229427,"[165, 0, 0, 0]","[572, 559, 546, 533]","[28.846153846153847, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
3,4.724900,4.057893,10.446112,1.385992,7.014250,9.656221,42.000000,0.229427,"[165, 0, 0, 0]","[572, 559, 546, 533]","[28.846153846153847, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
4,4.453000,3.977187,10.446112,1.385992,7.014250,9.656221,42.000000,0.229427,"[165, 0, 0, 0]","[572, 559, 546, 533]","[28.846153846153847, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
5,4.220500,3.914783,10.585361,1.478815,7.138204,9.795921,42.000000,0.228377,"[162, 0, 0, 0]","[572, 559, 546, 533]","[28.321678321678323, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
6,4.141700,3.859205,10.808095,1.487788,7.255901,9.883290,42.000000,0.229427,"[165, 0, 0, 0]","[572, 559, 546, 533]","[28.846153846153847, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
7,4.030400,3.813586,11.420675,1.724539,7.596066,10.126014,42.000000,0.230805,"[169, 0, 0, 0]","[572, 559, 546, 533]","[29.545454545454547, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
8,3.894100,3.770555,11.282747,1.440822,7.564672,9.983228,42.000000,0.230805,"[169, 0, 0, 0]","[572, 559, 546, 533]","[29.545454545454547, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
9,3.823700,3.729141,11.671462,1.888709,7.962214,10.666680,42.000000,0.232827,"[175, 0, 0, 0]","[572, 559, 546, 533]","[30.594405594405593, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
10,3.724300,3.692215,12.323524,1.885647,8.193222,11.184779,42.000000,0.230805,"[169, 0, 0, 0]","[572, 559, 546, 533]","[29.545454545454547, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13


***** Running Evaluation *****
  Num examples = 13
  Batch size = 8
Saving model checkpoint to tmp/model_output/checkpoint-1
Configuration saved in tmp/model_output/checkpoint-1/config.json
Model weights saved in tmp/model_output/checkpoint-1/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 13
  Batch size = 8
Saving model checkpoint to tmp/model_output/checkpoint-2
Configuration saved in tmp/model_output/checkpoint-2/config.json
Model weights saved in tmp/model_output/checkpoint-2/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 13
  Batch size = 8
Saving model checkpoint to tmp/model_output/checkpoint-3
Configuration saved in tmp/model_output/checkpoint-3/config.json
Model weights saved in tmp/model_output/checkpoint-3/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 13
  Batch size = 8
Saving model checkpoint to tmp/model_output/checkpoint-4
Configuration saved in tmp/model_output/checkpoint-4/config.json
Model weights saved in tmp/

TrainOutput(global_step=44, training_loss=3.398714845830744, metrics={'train_runtime': 435.3681, 'train_samples_per_second': 5.857, 'train_steps_per_second': 0.115, 'total_flos': 1358073635143680.0, 'train_loss': 3.398714845830744, 'epoch': 44.0})

In [ ]:
# Получение названия модели
model_name = trainer.model.__class__.__name__
print(f"Название обученной модели: {model_name}")

Название обученной модели: BartForConditionalGeneration


In [ ]:
 #predictions = trainer.predict(tokenized_val)
 #predictions

In [ ]:
df_gen = df.copy()

In [ ]:
# Обмен значениями столбцов "positive" и "negative"
df_gen['random_sentiment'] = df_gen['random_sentiment'].apply(lambda x: 'negative' if x=='positive' else 'positive' if x=='negative' else x)
# Вывод измененного датасета
df_gen.drop(columns = 'random')

,positive,negative,random_sentiment
0,"""Pixote: A Lei do Mais Fraco"" deals with what ...","And maybe, as Fred Sandford used to say, ""one ...",positive
1,"""Pixote: A Lei do Mais Fraco"" deals with what ...","The plot was very thin, although the idea of n...",negative
2,"""Pixote: A Lei do Mais Fraco"" deals with what ...",This is a typical late Universal Horror flick:...,negative
3,"""Pixote: A Lei do Mais Fraco"" deals with what ...","First of all, the actor they have to play Jesu...",negative
4,"""Pixote: A Lei do Mais Fraco"" deals with what ...","When i got this movie free from my job, along ...",positive
...,...,...,...
59,This is a cartoon series where most of the act...,"First of all, the actor they have to play Jesu...",positive
60,This is a cartoon series where most of the act...,"When i got this movie free from my job, along ...",negative
61,This is a cartoon series where most of the act...,do not ever watch this film...it is the bigges...,positive
62,This is a cartoon series where most of the act...,"Even duller, if possible, than the original (I...",positive


In [ ]:
gen = ('Positive: ' + df.positive + '\n' + 'Negative: ' + df.negative + '\n' + df.random_sentiment.str.title()).values


In [ ]:
import datasets

# Define your data as dictionaries for each split
gen_data = {
    "text": gen,

}


# Convert the dictionaries to `Dataset` objects
gen_dataset = datasets.Dataset.from_dict(gen_data)

# Create a `DatasetDict` object containing the train, validation, and test sets
data_generation= datasets.DatasetDict({
    "generation": gen_dataset,
})

# Print the sizes of the resulting datasets
print(data_generation)

DatasetDict({
    generation: Dataset({
        features: ['text'],
        num_rows: 64
    })
})


In [ ]:
generation = data_generation['generation']

In [ ]:
def tokenizing_fn2(instances):
    encoded = tokenizer(instances["text"], truncation=True)
    return encoded

tokenized_gen = generation.map(tokenizing_fn2, batched=True).remove_columns(['text'])

tokenized_gen

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 64
})

In [ ]:
dtype = torch.long

In [ ]:
import torch
from datasets import Dataset
from torch.utils.data import DataLoader

In [ ]:

# define the tensor dtype and device
dtype = torch.long
device = torch.device('cpu')

# convert input_ids and attention_mask to PyTorch tensors
input_ids = torch.tensor(dataset['input_ids'], dtype=dtype, device=device)


# create a tuple of tensors to be used as input to a PyTorch model
input_tensors = (input_ids)

# print the shapes of the tensors
print(input_ids.shape)  # should print (64, max_seq_len)
print(attention_mask.shape)  # should print (64, max_seq_len)

NameError: ignored

In [ ]:
# define the tensor dtype and device
dtype = torch.long
device = torch.device('cpu')

# set the maximum sequence length
max_seq_len = 425

# define a function to pad sequences with zeros
def pad_sequence(sequence, max_seq_len):
    if len(sequence) < max_seq_len:
        return sequence + [0] * (max_seq_len - len(sequence))
    else:
        return sequence[:max_seq_len]

# pad the input_ids and attention_mask sequences
input_ids = [pad_sequence(seq, max_seq_len) for seq in tokenized_gen['input_ids']]


# convert input_ids and attention_mask to PyTorch tensors
input_ids = torch.tensor(input_ids, dtype=dtype, device=device)


# create a DataLoader object to iterate over the data
batch_size = 16
dataloader = DataLoader(tokenized_gen, batch_size=batch_size, shuffle=True)

# iterate over the data in batches
for batch in dataloader:
    # pad the input_ids and attention_mask sequences in the batch
    batch_input_ids = [pad_sequence(seq, max_seq_len) for seq in batch['input_ids']]

    
    # convert the padded sequences to PyTorch tensors
    batch_input_ids = torch.tensor(batch_input_ids, dtype=dtype, device=device)



RuntimeError: ignored

In [ ]:
input_ids = torch.tensor(tokenized_gen['input_ids'], device=device)

ValueError: ignored

In [ ]:
input_ids = tokenized_gen['input_ids']


In [ ]:
outputs = trainer.model.generate(input_ids, max_new_tokens=1024, min_new_tokens=10, early_stopping=True, num_beams=4)


AttributeError: ignored

In [ ]:

# Декодирование сгенерированного текста
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

In [ ]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 2e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 20),
        "per_device_train_batch_size": trial.suggest_int("per_device_train_batch_size", 4, 32),
        "weight_decay": trial.suggest_float("weight_decay", 1e-10, 0.1, log=True),
    }

def compute_objective(metrics):
    return metrics["eval_rouge1"]

In [ ]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 13.6 MB/s eta 0:00:00


In [ ]:
hyp_opt_result = trainer.hyperparameter_search(
    direction="maximize",
    hp_space=hp_space,
    backend="optuna",
    compute_objective=compute_objective
)

[I 2023-05-04 09:34:35,835] A new study created in memory with name: no-name-5d33d914-ad8c-48e8-8b06-7c90b1b4ed6e
Trial:
loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at cache/model/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.a243ed957122436adb0b8d8e9d20f896f45c174b6324d625ca0a20a84f72a910
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
 

eval/bp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/gen_len,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,█▇▆▅█▇▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/ref_len,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/rouge1,▁▁▁▁▁▁▁▁▁▁▃▅▆▆▆▅▅▇▆▅▇█▇▇▇▇▇▆▇▆▆▆▆▇▆▇▇▇▆▆
eval/rouge2,▁▁▁▁▁▁▁▁▁▁▃▃▅▅▆▆▄▅▅▅▇█▆▆▇█▇▆▇▆▇▆▆▇▆▆▇▇▇▆
eval/rougeL,▁▁▁▁▁▁▁▁▁▁▅▆▆▇▇▆▆▇▆▆▇█▇▇██▇▇█▇▇▆▆▇▆▆▇▇▇▆
eval/rougeLsum,▁▁▁▁▁▁▁▁▁▁▄▆▇▇▇▆▆█▆▆▇█▇▇██▇▇█▇▇▆▆▇▆▇▇▇▇▇
eval/runtime,▂▇▁▁▂▁▁▁▁▂▁▁▁▁▂▁▁▁▁▂▁▂▁▂█▁▁▁▁▂▁▂▂▁▂▁▂▁▁▁
eval/samples_per_second,▇▂██▇█▇██▇████▇████▇█▇█▇▁█▇▇█▇█▇▇▇▇█▇▇▇█
eval/score,▁▁▁▁▁▁▁▁▁▁▂▅▇▅▆▅▄▅▅▄▆▇▆▆▇█▇▆▇▆▆▇▇▇▇▇██▇▇


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: axyniia. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Score,Counts,Totals,Precisions,Bp,Sys Len,Ref Len
1,4.574700,6.514341,11.503080,1.666843,7.861348,9.726462,42.000000,0.228023,"[161, 0, 0, 0]","[572, 559, 546, 533]","[28.146853146853147, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
2,4.459800,4.289412,12.959276,2.173946,9.573843,11.799176,42.000000,0.233818,"[178, 0, 0, 0]","[572, 559, 546, 533]","[31.11888111888112, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
3,3.921900,4.246862,15.830764,3.559961,11.329078,15.594827,42.000000,0.232493,"[174, 0, 0, 0]","[572, 559, 546, 533]","[30.41958041958042, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
4,3.721400,3.956554,18.005827,3.253040,11.618448,16.778394,42.000000,0.240126,"[198, 0, 0, 0]","[572, 559, 546, 533]","[34.61538461538461, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
5,3.404200,3.799709,18.126746,3.720300,12.223745,16.704690,42.000000,0.244550,"[213, 0, 0, 0]","[572, 559, 546, 533]","[37.23776223776224, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
6,3.142400,3.651707,17.917695,4.399891,11.559509,15.478553,42.000000,0.243974,"[211, 0, 0, 0]","[572, 559, 546, 533]","[36.88811188811189, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
7,2.996400,3.536949,17.857432,3.404298,11.069496,16.430982,42.000000,0.244550,"[213, 0, 0, 0]","[572, 559, 546, 533]","[37.23776223776224, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
8,2.805800,3.436662,18.385365,4.460152,11.636840,16.739935,42.000000,0.248746,"[228, 0, 0, 0]","[572, 559, 546, 533]","[39.86013986013986, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
9,2.760600,3.347263,17.647232,4.675047,11.396313,16.495061,42.000000,0.242516,"[206, 0, 0, 0]","[572, 559, 546, 533]","[36.01398601398601, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
10,2.713100,3.266997,17.671113,4.251426,11.052562,15.345017,42.000000,0.246815,"[221, 0, 0, 0]","[572, 559, 546, 533]","[38.63636363636363, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13


***** Running Evaluation *****
  Num examples = 13
  Batch size = 8
Saving model checkpoint to tmp/model_output/run-0/checkpoint-1
Configuration saved in tmp/model_output/run-0/checkpoint-1/config.json
Model weights saved in tmp/model_output/run-0/checkpoint-1/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 13
  Batch size = 8
Saving model checkpoint to tmp/model_output/run-0/checkpoint-2
Configuration saved in tmp/model_output/run-0/checkpoint-2/config.json
Model weights saved in tmp/model_output/run-0/checkpoint-2/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 13
  Batch size = 8
Saving model checkpoint to tmp/model_output/run-0/checkpoint-3
Configuration saved in tmp/model_output/run-0/checkpoint-3/config.json
Model weights saved in tmp/model_output/run-0/checkpoint-3/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 13
  Batch size = 8
Saving model checkpoint to tmp/model_output/run-0/checkpoint-4
Configuration saved in tmp/model

...

eval/bp,▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/gen_len,▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,█▃▄▃▃▂▂▂▁▁▁▁▁
eval/ref_len,▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/rouge1,▁▄▅█▇▇█▇▆▅▆▆▆
eval/rouge2,▁▃▆█▅▇▇▇▇▆▇▅▆
eval/rougeL,▁▅▆█▇▇▆▆▅▄▇▆▇
eval/rougeLsum,▁▅▆▇███▇▆▄▇▆▇
eval/runtime,▁▁▁▂▁▂▂▁█▁▂▂▁
eval/samples_per_second,█▇█▇█▇▇█▁█▇▇█
eval/score,▃▄▁▆▇▇▇█▇▇███


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Score,Counts,Totals,Precisions,Bp,Sys Len,Ref Len
1,4.574700,6.514341,11.503080,1.666843,7.861348,9.726462,42.000000,0.228023,"[161, 0, 0, 0]","[572, 559, 546, 533]","[28.146853146853147, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13
2,4.459800,4.266658,12.294363,2.097540,8.919603,10.969069,42.000000,0.232493,"[174, 0, 0, 0]","[572, 559, 546, 533]","[30.41958041958042, 0.08944543828264759, 0.045787545787545784, 0.02345215759849906]",1.000000,572,13


***** Running Evaluation *****
  Num examples = 13
  Batch size = 8
Saving model checkpoint to tmp/model_output/run-8/checkpoint-1
Configuration saved in tmp/model_output/run-8/checkpoint-1/config.json
Model weights saved in tmp/model_output/run-8/checkpoint-1/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 13
  Batch size = 8
[W 2023-05-04 09:46:26,083] Trial 8 failed with parameters: {'learning_rate': 7.368874648991927e-05, 'num_train_epochs': 9, 'per_device_train_batch_size': 17, 'weight_decay': 2.491134592427919e-07} because of the following error: TypeError("unsupported operand type(s) for /: 'int' and 'NoneType'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations.py", line 156, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/lo

TypeError: ignored

In [ ]:
result = hyp_opt_result.hyperparameters
result

In [ ]:
torch.save(result, "result")
#torch.save(objective, "objective")